In [1]:
!which python


/Users/saiha/opt/anaconda3/envs/churn-model/bin/python


In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
#standard imports
import os
import pickle
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,cross_val_score,learning_curve
import sklearn

from src.localpath import *
from src.data.make_dataset import load_training_data
from src.models.train_model import *
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
AdaBoostClassifier,
GradientBoostingClassifier,
RandomForestClassifier,


)
from sklearn.inspection import plot_partial_dependence
from sklearn.metrics import roc_curve

In [4]:
#Configuration for Jupyter Notebooks
pd.options.display.max_columns=100
pd.options.display.max_rows=100

## Load Featurized  Data

In [10]:
X_train,y_train=load_training_data(final=True)

# Feature Multicolinearity

In [13]:
vif=[variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]


/Users/saiha/opt/anaconda3/envs/churn-model/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [14]:
zip (X_train.columns,vif),key: lambda x: x[1], reverse=True

[1.0029253451164717,
 1.1524490787143984,
 1.4622286104923994,
 1.3768800344471095,
 7.528269480250208,
 1795.2936129533036,
 1.2187874943183166,
 879.4071653278096,
 10.871762430587555,
 61.269312275148884,
 7.388728481143919,
 150.413015558894,
 inf,
 inf,
 6.478979774573932,
 inf,
 6.911783532524887,
 inf,
 6.990713871204519,
 inf,
 6.498500323508462,
 inf,
 24.573123044650014,
 inf,
 24.400188765894438,
 1.6328101814762768,
 2.6312318158444175,
 1.5579786757164678,
 1.9381234819755606,
 1.8500671539360292]